In [1]:
# Add "parameters" (jupyter-notebook) tag to this cell, to allow papermill to inject different parameters
from datetime import date
it=0  #Iteration of gridsearch
# to put it all in one folder by date, will be replaced by papermill
today=date.today() 
rdate=today.strftime("%Y-%m-%d")
rdate="2021-08-31"
# Training Params
best_runs = []
training_stats = {}
epochs = 15
loops = 5

In [2]:
# Parameters
it = 14
rdate = "2021-08-31"


In [3]:
# See if running on Colab (for setting the correct workdir and installing all dependencies)
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  g_colab = True
else:
  print('Not running on CoLab')
  g_colab = False

Not running on CoLab


In [4]:
# connect to drive
if g_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    %cd "/gdrive/MyDrive/1 Job/Product and Code/CogAlex 2.0/"

# Choose model
# Gridsearch parameters
from sklearn.model_selection import ParameterGrid

# Original results with "xlm_roberta_base"
grid = [{"model_name": ["xlm-roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new"], ["de_train_new"], ["de_val_new"],
                      ["en_train_new", "en_val_new"], ["en_train_new"], ["en_val_new"], 
                      ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]},
        {"model_name": ["xlm-roberta-large", "distilbert-base-multilingual-cased", "bert-base-multilingual-uncased", "bert-base-multilingual-cased", "roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]}]

pg = list(ParameterGrid(grid))
print(len(pg))
model_name = pg[it]["model_name"]
datasets = pg[it]["datasets"]
print(pg[it])

18
{'datasets': ['de_train_new', 'de_val_new', 'en_train_new', 'en_val_new'], 'model_name': 'distilbert-base-multilingual-cased'}


# Manually set parameters

In [5]:
# Savedir
import os
savedir = f"./{rdate}/averages/{model_name}_{it}_{datasets}"
if g_colab:
    workdir = os.getcwd()
else:
    workdir = os.getcwd().replace("/home/","/binfl/")
model_dir = f"{workdir}/saved_models/{rdate}"

# Check for GPU support
import torch
if torch.cuda.is_available():
        print("Using GPU for inference")
        print(torch.cuda.get_device_name(torch.cuda.current_device()))
        device = torch.cuda.current_device()
else:
        print("Using CPU for inference")
        device = -1

Using GPU for inference
GeForce GTX 1080


In [6]:
if g_colab:
    !pip install transformers
    !pip install sentencepiece

# Libraries

In [7]:
import torch                                              #for training the model
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
import pandas as pd                                       #for handling the data
from transformers import XLMRobertaTokenizer, AutoTokenizer              #for loading the pretrained model and tokenizer
from transformers import XLMRobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW                            
from transformers import get_linear_schedule_with_warmup
from transformers import pipeline
from sklearn import preprocessing                         #for label encoding
from sklearn.metrics import classification_report         #for showing performance on validation/test sets
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid         #for grid search
from sklearn.model_selection import ParameterSampler      #for random search
from sklearn.utils.fixes import loguniform
import sentencepiece
import matplotlib.pyplot as plt
import time
import datetime
import random
import numpy as np
%matplotlib inline
import seaborn as sns
# For file saving etc.
import os
import shutil

# Load Data

In [8]:
# sk learn label encoder for changing the labels to integers
labels=["ANT", "HYP", "RANDOM", "SYN"]
le = preprocessing.LabelEncoder()
le.fit(labels)
print(labels)

['ANT', 'HYP', 'RANDOM', 'SYN']


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
if "gpt2" in model_name:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token_id

0

# Evaluate Validation

In [10]:
label_dict = {i : l for i,l in enumerate(labels)}
print(label_dict)

{0: 'ANT', 1: 'HYP', 2: 'RANDOM', 3: 'SYN'}


In [11]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "all":0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "all":""}
for run in os.listdir(model_dir):
    print(run + "\n")
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",
                                                               num_labels=4,  
                                                               id2label=label_dict)                                                        
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    validgold_zh, validgold_de, validgold_en = [open(f"./datasets/validgold_{l}_data.txt").readlines() for l in ["chinese", "german", "english"]]
    validgold_all =   validgold_zh + validgold_de + validgold_en 
    for data, read_lang in zip([validgold_zh, validgold_de, validgold_en, validgold_all], ["chinese", "german", "english", "all"]):
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict

Run 3



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.74      0.83      0.78       136
         HYP       0.79      0.78      0.78       145
      RANDOM       0.92      0.89      0.90       428
         SYN       0.80      0.80      0.80       129

    accuracy                           0.85       838
   macro avg       0.81      0.82      0.82       838
weighted avg       0.85      0.85      0.85       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.85      0.83      0.84       136
         HYP       0.91      0.78      0.84       145
         SYN       0.87      0.80      0.83       129

   micro avg       0.88      0.80      0.84       410
   macro avg       0.88      0.80      0.84       410
weighted avg       0.88      0.80      0.84       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.31      0.20      0.25       275
         HYP       0.42      0.21      0.28       294
      RANDOM       0.56      0.78      0.65       786
         SYN       0.27      0.19      0.23       272

    accuracy                           0.48      1627
   macro avg       0.39      0.35      0.35      1627
weighted avg       0.44      0.48      0.44      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.49      0.20      0.29       275
         HYP       0.59      0.21      0.31       294
         SYN       0.40      0.19      0.26       272

   micro avg       0.48      0.20      0.29       841
   macro avg       0.49      0.20      0.29       841
weighted avg       0.49      0.20      0.29       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.38      0.26      0.31       308
         HYP       0.43      0.38      0.40       292
      RANDOM       0.67      0.77      0.71       877
         SYN       0.35      0.34      0.34       259

    accuracy                           0.55      1736
   macro avg       0.46      0.44      0.44      1736
weighted avg       0.53      0.55      0.54      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.58      0.26      0.36       308
         HYP       0.56      0.38      0.45       292
         SYN       0.48      0.34      0.40       259

   micro avg       0.54      0.32      0.40       859
   macro avg       0.54      0.33      0.40       859
weighted avg       0.54      0.32      0.40       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.46      0.35      0.40       719
         HYP       0.53      0.39      0.45       731
      RANDOM       0.66      0.80      0.72      2091
         SYN       0.42      0.37      0.39       660

    accuracy                           0.58      4201
   macro avg       0.52      0.48      0.49      4201
weighted avg       0.56      0.58      0.57      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.65      0.35      0.45       719
         HYP       0.67      0.39      0.49       731
         SYN       0.56      0.37      0.45       660

   micro avg       0.62      0.37      0.46      2110
   macro avg       0.63      0.37      0.46      2110
weighted avg       0.63      0.37      0.46      2110

Run 5



...

Results for all: 

              precision    recall  f1-score   support

         ANT       0.46      0.35      0.40       719
         HYP       0.53      0.39      0.45       731
      RANDOM       0.66      0.80      0.72      2091
         SYN       0.42      0.37      0.39       660

    accuracy                           0.58      4201
   macro avg       0.52      0.48      0.49      4201
weighted avg       0.56      0.58      0.57      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.65      0.35      0.45       719
         HYP       0.67      0.39      0.49       731
         SYN       0.56      0.37      0.45       660

   micro avg       0.62      0.37      0.46      2110
   macro avg       0.63      0.37      0.46      2110
weighted avg       0.63      0.37      0.46      2110

Run 1



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.85      0.81      0.83       136
         HYP       0.83      0.79      0.81       145
      RANDOM       0.91      0.93      0.92       428
         SYN       0.80      0.83      0.82       129

    accuracy                           0.87       838
   macro avg       0.85      0.84      0.84       838
weighted avg       0.87      0.87      0.87       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.91      0.81      0.86       136
         HYP       0.91      0.79      0.85       145
         SYN       0.87      0.83      0.85       129

   micro avg       0.90      0.81      0.85       410
   macro avg       0.90      0.81      0.85       410
weighted avg       0.90      0.81      0.85       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.26      0.26      0.26       275
         HYP       0.32      0.24      0.28       294
      RANDOM       0.60      0.62      0.61       786
         SYN       0.22      0.25      0.24       272

    accuracy                           0.43      1627
   macro avg       0.35      0.35      0.35      1627
weighted avg       0.43      0.43      0.43      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.41      0.26      0.32       275
         HYP       0.49      0.24      0.33       294
         SYN       0.36      0.25      0.30       272

   micro avg       0.41      0.25      0.31       841
   macro avg       0.42      0.25      0.31       841
weighted avg       0.42      0.25      0.31       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.37      0.38      0.37       308
         HYP       0.37      0.36      0.37       292
      RANDOM       0.70      0.71      0.70       877
         SYN       0.37      0.35      0.36       259

    accuracy                           0.54      1736
   macro avg       0.45      0.45      0.45      1736
weighted avg       0.54      0.54      0.54      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.59      0.38      0.46       308
         HYP       0.51      0.36      0.42       292
         SYN       0.48      0.35      0.41       259

   micro avg       0.53      0.36      0.43       859
   macro avg       0.53      0.36      0.43       859
weighted avg       0.53      0.36      0.43       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.41      0.41      0.41       719
         HYP       0.45      0.40      0.42       731
      RANDOM       0.70      0.72      0.71      2091
         SYN       0.39      0.40      0.39       660

    accuracy                           0.56      4201
   macro avg       0.49      0.48      0.49      4201
weighted avg       0.56      0.56      0.56      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.60      0.41      0.49       719
         HYP       0.61      0.40      0.48       731
         SYN       0.53      0.40      0.46       660

   micro avg       0.58      0.41      0.48      2110
   macro avg       0.58      0.41      0.48      2110
weighted avg       0.58      0.41      0.48      2110



In [12]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results.loc[k] = temp_df.mean()
avg_val_results.update(avg_val_results.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results

,precision,recall,f1-score,support
chinese,85.3,85.1,85.2,838.0
german,44.0,47.2,44.1,1627.0
english,53.0,54.8,53.6,1736.0
all,56.3,57.9,56.6,4201.0


In [13]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results_no_random.loc[k] = temp_df.mean()
avg_val_results_no_random.update(avg_val_results_no_random.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results_no_random

,precision,recall,f1-score,support
chinese,88.1,80.4,84.0,410.0
german,48.0,21.2,29.2,841.0
english,54.1,33.3,40.9,859.0
all,61.9,37.6,46.7,2110.0


In [14]:
# save average scores
os.makedirs(savedir, exist_ok=True)
val_preds_savedir = savedir + "/val_preds"
os.makedirs(val_preds_savedir, exist_ok=True)
avg_val_results.to_csv(f"{savedir}/val_avg_{model_name}_{it}.csv")
avg_val_results_no_random.to_csv(f"{savedir}/val_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(val_preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        

# Evaluate on Gold

In [15]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "italian":0, "german NEW":0, "english NEW": 0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "italian":"", "german NEW":"", "english NEW": ""}
for run in os.listdir(model_dir):
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",  id2label=label_dict)
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    gold_zh, gold_de, gold_en, gold_it = [open(f"./datasets/gold_{l}_data.txt").readlines() for l in ["chinese", "german", "english", "italian"]]
    gold_de_new, gold_en_new = [open(f"./datasets/gold_{l}_data_new.txt").readlines() for l in ["german", "english"]]   
    for data, read_lang in zip([gold_zh, gold_de, gold_en, gold_it, gold_de_new, gold_en_new], ["chinese", "german", "english", "italian", "german NEW", "english NEW"]):
        # Load data for prediction/scoring
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        
        #Predict labels on test_set
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict
        #print(results_no_random[run][read_lang]["weighted avg"])

Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.75      0.80      0.78       142
         HYP       0.73      0.81      0.76       129
      RANDOM       0.93      0.88      0.90       445
         SYN       0.80      0.80      0.80       122

    accuracy                           0.84       838
   macro avg       0.80      0.82      0.81       838
weighted avg       0.85      0.84      0.84       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.86      0.80      0.83       142
         HYP       0.89      0.81      0.85       129
         SYN       0.87      0.80      0.83       122

   micro avg       0.87      0.80      0.84       393
   macro avg       0.87      0.80      0.84       393
weighted avg       0.87      0.80      0.84       393



Results for german: 

              precision    recall  f1-score   support

         ANT       0.33      0.20      0.25       281
         HYP       0.36      0.17      0.23       286
      RANDOM       0.58      0.80      0.67       796
         SYN       0.31      0.26      0.28       265

    accuracy                           0.50      1628
   macro avg       0.39      0.36      0.36      1628
weighted avg       0.45      0.50      0.46      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.47      0.20      0.28       281
         HYP       0.51      0.17      0.26       286
         SYN       0.45      0.26      0.33       265

   micro avg       0.47      0.21      0.29       832
   macro avg       0.48      0.21      0.29       832
weighted avg       0.48      0.21      0.29       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.38      0.29      0.33       306
         HYP       0.38      0.35      0.36       279
      RANDOM       0.67      0.76      0.71       887
         SYN       0.34      0.32      0.33       266

    accuracy                           0.54      1738
   macro avg       0.44      0.43      0.43      1738
weighted avg       0.52      0.54      0.53      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.55      0.29      0.38       306
         HYP       0.55      0.35      0.43       279
         SYN       0.45      0.32      0.37       266

   micro avg       0.51      0.32      0.39       851
   macro avg       0.52      0.32      0.39       851
weighted avg       0.52      0.32      0.39       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.19      0.31      0.23       144
         HYP       0.29      0.18      0.22       153
      RANDOM       0.65      0.46      0.53       523
         SYN       0.22      0.37      0.28       187

    accuracy                           0.38      1007
   macro avg       0.34      0.33      0.32      1007
weighted avg       0.45      0.38      0.40      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.34      0.31      0.32       144
         HYP       0.54      0.18      0.27       153
         SYN       0.40      0.37      0.38       187

   micro avg       0.40      0.29      0.33       484
   macro avg       0.43      0.28      0.32       484
weighted avg       0.42      0.29      0.33       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.33      0.20      0.25       281
         HYP       0.40      0.30      0.35       286
      RANDOM       0.61      0.78      0.68       796
         SYN       0.32      0.26      0.29       265

    accuracy                           0.51      1628
   macro avg       0.41      0.39      0.39      1628
weighted avg       0.48      0.51      0.49      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.48      0.20      0.29       281
         HYP       0.56      0.30      0.39       286
         SYN       0.43      0.26      0.32       265

   micro avg       0.49      0.26      0.34       832
   macro avg       0.49      0.26      0.33       832
weighted avg       0.49      0.26      0.34       832



...

Results for german: 

              precision    recall  f1-score   support

         ANT       0.31      0.31      0.31       281
         HYP       0.32      0.25      0.28       286
      RANDOM       0.59      0.61      0.60       796
         SYN       0.25      0.29      0.27       265

    accuracy                           0.44      1628
   macro avg       0.37      0.36      0.36      1628
weighted avg       0.44      0.44      0.44      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.51      0.31      0.38       281
         HYP       0.50      0.25      0.34       286
         SYN       0.41      0.29      0.34       265

   micro avg       0.47      0.28      0.35       832
   macro avg       0.47      0.28      0.35       832
weighted avg       0.47      0.28      0.35       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.39      0.40      0.39       306
         HYP       0.36      0.38      0.37       279
      RANDOM       0.70      0.68      0.69       887
         SYN       0.32      0.32      0.32       266

    accuracy                           0.53      1738
   macro avg       0.44      0.44      0.44      1738
weighted avg       0.53      0.53      0.53      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.60      0.40      0.48       306
         HYP       0.53      0.38      0.44       279
         SYN       0.46      0.32      0.37       266

   micro avg       0.53      0.37      0.43       851
   macro avg       0.53      0.36      0.43       851
weighted avg       0.53      0.37      0.43       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.20      0.35      0.25       144
         HYP       0.24      0.23      0.23       153
      RANDOM       0.70      0.38      0.49       523
         SYN       0.23      0.40      0.29       187

    accuracy                           0.35      1007
   macro avg       0.34      0.34      0.32      1007
weighted avg       0.47      0.35      0.38      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.36      0.35      0.35       144
         HYP       0.43      0.23      0.30       153
         SYN       0.41      0.40      0.40       187

   micro avg       0.40      0.33      0.36       484
   macro avg       0.40      0.32      0.35       484
weighted avg       0.40      0.33      0.36       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.32      0.32      0.32       281
         HYP       0.40      0.41      0.40       286
      RANDOM       0.64      0.61      0.62       796
         SYN       0.29      0.31      0.30       265

    accuracy                           0.48      1628
   macro avg       0.41      0.41      0.41      1628
weighted avg       0.48      0.48      0.48      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.51      0.32      0.39       281
         HYP       0.61      0.41      0.49       286
         SYN       0.44      0.31      0.37       265

   micro avg       0.52      0.35      0.42       832
   macro avg       0.52      0.35      0.42       832
weighted avg       0.52      0.35      0.42       832



Results for english NEW: 

              precision    recall  f1-score   support

         ANT       0.41      0.42      0.41       306
         HYP       0.44      0.47      0.46       283
      RANDOM       0.73      0.70      0.71       887
         SYN       0.36      0.39      0.37       262

    accuracy                           0.56      1738
   macro avg       0.49      0.49      0.49      1738
weighted avg       0.57      0.56      0.57      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.64      0.42      0.50       306
         HYP       0.61      0.47      0.53       283
         SYN       0.50      0.39      0.44       262

   micro avg       0.58      0.43      0.49       851
   macro avg       0.58      0.43      0.49       851
weighted avg       0.59      0.43      0.49       851



In [16]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results.loc[k] = temp_df.mean()
avg_test_results.update(avg_test_results.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results

,precision,recall,f1-score,support
chinese,85.1,84.6,84.7,838.0
german,44.9,48.6,45.3,1628.0
english,52.4,53.9,53.0,1738.0
italian,45.3,37.1,39.2,1007.0
german NEW,47.8,50.5,48.4,1628.0
english NEW,55.4,56.5,55.8,1738.0


In [17]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results_no_random.loc[k] = temp_df.mean()
avg_test_results_no_random.update(avg_test_results_no_random.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results_no_random

,precision,recall,f1-score,support
chinese,87.8,80.6,84.0,393.0
german,47.6,22.2,29.9,832.0
english,52.1,32.7,40.1,851.0
italian,42.0,29.7,33.3,484.0
german NEW,49.8,27.5,35.2,832.0
english NEW,56.0,37.2,44.5,851.0


In [18]:
# save average scores
os.makedirs(savedir, exist_ok=True)
preds_savedir = savedir + "/preds"
os.makedirs(preds_savedir, exist_ok=True)
avg_test_results.to_csv(f"{savedir}/test_avg_{model_name}_{it}.csv")
avg_test_results_no_random.to_csv(f"{savedir}/test_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        